# Initialization

In [1]:
# Google Earth Engine packages
import ee
import geemap
import geemap.colormaps as cm

import xarray as xr
import numpy as np


# function to load nc file into GEE
def netcdf_to_ee(filename):
    ds = xr.open_dataset(filename)
    data = ds['z']

    lon_data = np.round(data['lon'], 3)
    lat_data = np.round(data['lat'], 3)

    dim_lon = np.unique(np.ediff1d(lon_data).round(3))
    dim_lat = np.unique(np.ediff1d(lat_data).round(3))

    if (len(dim_lon) != 1) or (len(dim_lat) != 1):
        print("The netCDF file is not a regular longitude/latitude grid")

    data_np = np.array(data)
    data_np = np.transpose(data_np)

    # Figure out if we need to roll the data or not
    # (see https://github.com/giswqs/geemap/issues/285#issuecomment-791385176)
    if np.max(lon_data) > 180:
        data_np = np.roll(data_np, 180, axis=0)
        west_lon = lon_data[0] - 180
    else:
        west_lon = lon_data[0]

    transform = [dim_lon[0], 0, float(west_lon) - dim_lon[0]/2, 0, dim_lat[0], float(lat_data[0]) - dim_lat[0]/2]

    image = geemap.numpy_to_ee(
        data_np, "EPSG:4326", transform=transform, band_names='z'
    )
    return image, data_np, transform

In [2]:
# initialize GEE at the beginning of session
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()         # authenticate when using GEE for the first time
    ee.Initialize()
    
Map = geemap.Map()

In [3]:
import configparser
import ast

# read local config.ini file
config = configparser.ConfigParser()
config.read('config.ini')

# get file config from config.ini
dir_input = config['FILE_SETTINGS']['DIR_INPUT']
dir_output = config['FILE_SETTINGS']['DIR_OUTPUT']
output_gpkg = dir_output + config['FILE_SETTINGS']['GPKG_NAME']

# get date range for forcing data
date_range = ast.literal_eval(config['CONFIG']['DATE_RANGE'])

***

# ERA5L Geopotential

### Load input data

Load ERA5L file for geopotential and add to Map

In [4]:
file = 'ERA5_land_Z_geopotential_HMA.nc'
filename = dir_input + file

In [5]:
image, data_np, transform = netcdf_to_ee(filename)

In [6]:
# add image as layer
vis_params =  {'min': int(data_np.min()), 'max': int(data_np.max()), 'palette': cm.palettes.terrain, 'opacity': 0.8}
Map.addLayer(image, vis_params, "ERA5L geopotential")

Load catchment and add to map

In [7]:
import geopandas as gpd

catchment_new = gpd.read_file(output_gpkg, layer='catchment_new')

catchment = geemap.geopandas_to_ee(catchment_new)
Map.addLayer(catchment, {'color': 'darkgrey'}, "Catchment")
Map.centerObject(catchment, zoom=9)

Show Map

In [8]:
Map

Map(center=[42.183061016411926, 78.19022020297338], controls=(WidgetControl(options=['position', 'transparent_…

### Calculate weighted average geopotential and convert to elevation

In [9]:
# execute reducer
dict = image.reduceRegion(ee.Reducer.mean(),
                          geometry=catchment,
                          crs='EPSG:4326',
                          crsTransform=transform)

# get mean value and print
mean_val = dict.getInfo()['z']
ele_dat = mean_val / 9.80665
print(f'geopotential mean: {mean_val:.2f}, elevation: {ele_dat:.2f}')

geopotential mean: 32731.77, elevation: 3337.71


***

### ERA5L Temperature and Precipitation Time Series

In [10]:
collection = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_RAW')\
    .select('temperature_2m','total_precipitation_sum')\
    .filterDate(date_range[0], date_range[1])

In [11]:
def setProperty(image):
    dict = image.reduceRegion(ee.Reducer.mean(), catchment)
    return image.set(dict)


withMean = collection.map(setProperty)

In [12]:
import pandas as pd
import datetime

df = pd.DataFrame()
df['ts'] = withMean.aggregate_array('system:time_start').getInfo()
df['temp'] = withMean.aggregate_array('temperature_2m').getInfo()
df['temp_c'] = df['temp'] - 273.15
df['prec'] = withMean.aggregate_array('total_precipitation_sum').getInfo()
df['prec'] = df['prec'] * 1000
df['dt'] = df['ts'].apply(lambda x: datetime.datetime.fromtimestamp(x / 1000))

In [13]:
df.to_csv(dir_output + 'ERA5L.csv',header=True,index=False)

In [14]:
display(df)

,ts,temp,temp_c,prec,dt
0,378691200000,254.942000,-18.208000,0.060925,1982-01-01 01:00:00
1,378777600000,253.960229,-19.189771,0.013262,1982-01-02 01:00:00
2,378864000000,254.730269,-18.419731,0.111354,1982-01-03 01:00:00
3,378950400000,257.254092,-15.895908,0.504703,1982-01-04 01:00:00
4,379036800000,260.315701,-12.834299,1.016434,1982-01-05 01:00:00
...,...,...,...,...,...
3646,693705600000,252.339351,-20.810649,0.010638,1991-12-26 01:00:00
3647,693792000000,253.434769,-19.715231,0.008999,1991-12-27 01:00:00
3648,693878400000,252.528328,-20.621672,0.016598,1991-12-28 01:00:00
3649,693964800000,254.197170,-18.952830,0.058155,1991-12-29 01:00:00


Append data reference elevation to settings.yml.

In [15]:
import yaml

def read_yaml(file_path):
    with open(file_path, 'r') as f:
        data = yaml.safe_load(f)
        return data
    
def write_yaml(data, file_path):
    with open(file_path, 'w') as f:
        yaml.safe_dump(data, f)

def update_yaml(file_path, new_items):
    data = read_yaml(file_path)
    data.update(new_items)
    write_yaml(data, file_path)

        
update_yaml(dir_output + 'settings.yml', {'ele_dat': float(ele_dat)})

# Ab hier wird es experimentell...

In [9]:
collection = ee.ImageCollection('NASA/GDDP-CMIP6')\
    .select('tas','pr')\
    .filterDate('2020-01-01', '2020-12-31')\
    .filter(ee.Filter.eq('model', 'ACCESS-CM2'))\
    .filter(ee.Filter.eq('scenario', 'ssp245'))
    

In [10]:
import pandas as pd
import datetime

withMean = collection.map(setProperty)

df = pd.DataFrame()


In [11]:
array = ee.Array([withMean.aggregate_array('tas'),withMean.aggregate_array('pr')])

In [12]:
x = array.getInfo()

In [13]:
df['ts'] = withMean.aggregate_array('system:time_start').getInfo()
df['temp'] = withMean.aggregate_array('tas').getInfo()
df['temp_corr_c'] = df['temp'] - 273.15
df['prec'] = withMean.aggregate_array('pr').getInfo()
df['dt'] = df['ts'].apply(lambda x: datetime.datetime.fromtimestamp(x / 1000))

In [ ]:
df.to_csv(dir_output + 'ACCESS-CM2_ssp245.csv',header=True,index=False)

In [14]:
display(df)

,ts,temp,temp_corr_c,prec,dt
0,1577836800000,258.333397,-14.816603,0.000000,2020-01-01 01:00:00
1,1577923200000,258.416608,-14.733392,0.000008,2020-01-02 01:00:00
2,1578009600000,257.234564,-15.915436,0.000000,2020-01-03 01:00:00
3,1578096000000,257.343407,-15.806593,0.000000,2020-01-04 01:00:00
4,1578182400000,258.467121,-14.682879,0.000001,2020-01-05 01:00:00
...,...,...,...,...,...
360,1608940800000,254.425959,-18.724041,0.000000,2020-12-26 01:00:00
361,1609027200000,255.401343,-17.748657,0.000000,2020-12-27 01:00:00
362,1609113600000,256.649523,-16.500477,0.000009,2020-12-28 01:00:00
363,1609200000000,254.151730,-18.998270,0.000000,2020-12-29 01:00:00


In [17]:
dataset = ee.ImageCollection('NASA/GDDP-CMIP6')\
                .filter(ee.Filter.eq('model', 'ACCESS-CM2'))\
                .filter(ee.Filter.date('2014-07-01', '2014-07-02'))
minimumAirTemperature = dataset.select('tasmin')
minimumAirTemperatureVis = {
  'min': 240,
  'max': 310,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}
Map.setCenter(71, 52, 3)
Map.addLayer(minimumAirTemperature, minimumAirTemperatureVis, 'Minimum Air Temperature (K)')

In [16]:
Map

Map(center=[52, 71], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [ ]:
dataset